In [1]:
import math # for log
import gmpy2 # for mod_inverse

In [2]:
# Global Constants
DISTINCT = 37
S_ID = 2 # Self Id
R_ID = 1 # Retrieval sender Id
CHECK = ord('-')-ord('a') # for '-'
FIX = 26 # for '-'
FIX_NUM = 27 # for number '0' and so on...

In [3]:
# Load final message from file which has been transferred over unsafe network
RCVD_MSG = open('send_over_unsafe_network.txt', 'r').read()

In [4]:
# CA Details
CA_e, CA_n = map(int, open('public_CA.txt', 'r').read().split())

In [5]:
# Find dynamic block size by formula mentioned in class
def find_block_size(num_diff_chars, n):
    block_size = int(math.log(n, num_diff_chars))

    if num_diff_chars**block_size >= n:
        return block_size-1
    else:
        return block_size

In [6]:
# RSA Dencryption : Using CRT if possible
def decrypt_RSA(ci_text, key, n, CRT='no', p=None, q=None):
    pl_text = ''
    r = find_block_size(DISTINCT, n)
    s = r+1
    
    for i in range(0, len(ci_text), s):
        block = ci_text[i:i+s]
        C = 0
        for j, ch in enumerate(block[::-1]): # Reverse of encryption
            if ch=='-':
                C += (FIX)*(DISTINCT**j)
            elif '0' <= ch <= '9':
                C += (ord(ch) - ord('0') + FIX_NUM)*(DISTINCT**j)
            else:
                C += (ord(ch) - ord('a'))*(DISTINCT**j)
        
        
        # ====================================
        if CRT=='no':
            M = pow(C, key, n)
        else:
            dP = key % (p-1)
            dQ = key % (q-1)

            g, ss, t = gmpy2.gcdext(q, p)
            q_inv = int(ss) % p

            x1 = pow(C, dP, p)
            x2 = pow(C, dQ, q)
            h = q_inv*(x1-x2) % p
            M = (x2 + h*q)
        # ====================================
    
        updated_block = ''
        for j in range(r-1,-1,-1):
            temp = M // (DISTINCT**j)
            M -= temp*(DISTINCT**j)
            
            # for Numbers
            if FIX_NUM <= temp <= FIX_NUM+9:
                updated_block += chr(temp-FIX_NUM + ord('0'))
            else:
                if temp == FIX: temp = CHECK # for '-'
                updated_block += chr(temp+ord('a'))
            
        pl_text += updated_block[::-1]
    
    return pl_text

In [7]:
# To remove ('a'=0) appended while decrypting RSA
def remove_extra_a(with_a):
    return with_a.rstrip('a')

In [8]:
# Self Details : e, d, p, q
S_keys = open('private_' + str(S_ID) + '.txt', 'r').read() # Encrypted by CA
S_keys

'gaee-hv07b-qsy0z-matw5co0tx-s7v-00-2trt43j6fb7fomxxzz85qx98igh4056ye7qy90crfw9neb80vjec7w6kcahoyg02cfxoq603ogdfqx1-e73639t8anakd9xrcw0u7-4-im-mn03kkc2pq7l7l18jemgyiy413bwclcb48t4aw69cgzqvc-l1x95wvqnibf32kt3n6qfsf7bzd2u1uf4ybwyvlgoes7plnujz45-8y-r5uo41l8dmvymah1zf161hh596b4z4-2i1yojtji4kqaa-3gbvnxayi5uhvbb68qd-53gt1txwubk8n5qtzuuvejk1q1x-n0af7tv2y2y-crwanfnfazl49-z33z9uigaqrvrbavih44t-tzc8bpkqib9e65g786ubsqj3gaiydbdlbt6zngkt9l2l1wl-by0rzayoegabpl7c9n4wuqeum9h5f0i9qqjngyvnimxh454hc0uh6rivsh879yyrm3q6xksfz9la6iz87303gvt2egu04r0cl5vhc9tzt8cy1apfq3ddyvyxovf8bgw81tkgvbvrz-qd9lda6dj3nzhbsddgh8h0gzv-b4-ugp07q1y3k6n6krgimbmk6czf820nqw65lh19wp4a5f5ghz7tr-snx12ux993itrm3sdcgm-jebjmi-po4qifxm9t7nvded-f8ql3suqzg51qo3ksk9t8nla6ofzg561v1qmd2rcc2bwvm4-ac4jg7d38ozhofdtzs2gcxkdnpzzbzl-hezsutceipdxxh92t4xft3bx73bmfsijj2oscvjrgjk3mu-wno0-eegge3k670rrhiqzramsvfeohc80kwz3rvd7-1terkjvbw7-9ur7voiiy96mzs0lfp45xwhhq60h0bn6pb7ev96xuhjvegfp1fd5lljl608nti4hyl48jdaov0ok-5urg61kt3o11dgkbbs8ji7qiyso4pq'

In [9]:
S_e, S_d, S_p, S_q = map(int, list(map(remove_extra_a, decrypt_RSA(S_keys, CA_e, CA_n).split('-'))))
S_n = S_p * S_q
S_e, S_p

(33184838627527307195292164418122878877531195366784288356082933114097018992711228691483108302612936701519148129135989406978290848764998498532474647241692881312990339059669327387212114156400220747001909430936408996588018429519506206371965187396869279498796340851221097563184573791247340175228689441270603021769,
 16462028188369466240563303199086254917953432076523855802512125871682339058517056369717127400141615951240677491201965905200255260525407550533075764055908687)

In [10]:
# Find Receiver's Details : e, n
for row in open('public_dir.txt', 'r').read().split('\n')[:-1]:
    row_details = row.split(' ')
    if int(row_details[0]) == R_ID:
        R_keys = row_details[1]
        R_e, R_n = map(int, list(map(remove_extra_a, decrypt_RSA(R_keys, CA_e, CA_n).split('-'))))
R_e, R_n

(81500537003248282447372678957598622637375471670002623435102335944650549145718167993126832732363193409594784892195565191744196681706612311752022636975842364669876971098073740694165178107968419300938307981237082323012865192052566462123442221800953212883568621139771666619493672509370686908367914166725767604809,
 480463554232724092828465585279591064496111993593641642343777373333283580514525045643245435947071095553257018845607736949105128698641220056417419555389680045147058923322527697449012914887500324861680056054934993085328078107142619033627267957732031551889324582532531328809988756306261743167346072693219532120957)

In [11]:
# Remove RSA encryption of by self Private key    (CRT)
decrypt1_msg = decrypt_RSA(RCVD_MSG, S_d, S_n, 'yes', S_p, S_q).rstrip('a')
decrypt1_msg

'qpcphbxwrfhmbo22hr2fxd6f2wb1ny8y8q9-z57aqbo33mm0cjph5a3ekg927e113--b9fh9oye6w2zb4i0xq-3px87fkrnm68b0wd4gwmz7fjryi0lyi18m6i91h18kl-lzix7gwhiwtbna4i44xo26g6vtejwer9sk86wsc5mnjoucmje2ip3jx4yd537zsmkjk'

In [12]:
# Vig_key and Enc_Msg separated by '-'
key_vig, encrypt_vig = map(remove_extra_a, decrypt_RSA(decrypt1_msg, R_e, R_n).split('-'))
key_vig, encrypt_vig

('zsetcnbgwoxjdhlafvku',
 'gwpeqgioowpjelluydpokvervujydopkhlyahmktxjmwggikzsxaglaawooxzfhagefcaoonluehwzknnxghtbogrwodvdtlqtyoawrnoofx')

In [13]:
# Decrypt to get the original Msg by using Vig_key
def decrypt_vignere(ci_text, key):
    """
        Takes cipher_text(Lowercase Letters) and key as argument and returns decrypted plain_text.
    """
    pl_text = ''
    key_len = len(key)
    for i in range(len(ci_text)):
        pl_char = ( ord(ci_text[i]) - ord('a') - ord(key[i%key_len]) + ord('a') )%26
        pl_text += chr( pl_char + ord('a') )
    
    return pl_text

In [14]:
decrypt_vignere(encrypt_vig, key_vig)

'hellothisisabeautifuldaythishasbeenacrazyridethedeardepartedandhereweareinthreatofcoronaviruswillyoubenumber'

In [ ]:
# Fails at (3^5 mod 21)^e mod 15 = 0.. So can't recover back the message.
#          (M^S_d mod S_n)^R_e mod R_n 